# Distribution quantique de clés

**Alice:**

In [ ]:
from numpy import random

N = 32
alice_data_bits = random.randint(0,2,N)
alice_base_bits = random.randint(0,2,N)

print(alice_data_bits)
print(alice_base_bits)

In [ ]:
from qiskit.quantum_info import Statevector

alice_qubits = []
for i in range(N):
    data_bit = alice_data_bits[i]
    base_bit = alice_base_bits[i]
    if base_bit == 0:
        if (data_bit == 0): alice_qubits.append(Statevector.from_label('0'))
        else: alice_qubits.append(Statevector.from_label('1'))
    else:
        if (data_bit == 0): alice_qubits.append(Statevector.from_label('+'))
        else: alice_qubits.append(Statevector.from_label('-'))

for i in range(5):
    display(alice_qubits[i].draw('latex'))


**Alice:** Je t'envoie mes qubits...

**Bob:**

In [ ]:
bob_qubits = alice_qubits

print("J'ai bien reçu les qubits!")

Bob va maintenant mesurer alétoirement les qubits d'alice dans la base $Z$ our $X$, sans communiquer son choix de base.

-----------------------------

### Base Z:

Les états $\ket 0$ et $\ket 1$ restent inchangés par l'action de l'opérateur $Z$: 

\begin{equation*}
    Z\ket 0 = \ket 0    \qquad \text{et} \qquad    Z\ket 1 = \ket 1
\end{equation*}

Probabilités:

\begin{equation*}
    p(0) = |\langle 0|\psi\rangle|^2    \qquad \text{et} \qquad    p(1) = |\langle 1|\psi\rangle|^2
\end{equation*}

### Base X:


Les états $\ket +$ et $\ket -$ restent inchangés par l'action de l'opérateur $X$: 

\begin{equation*}
    X \frac{\ket 0 + \ket 1}{\sqrt{2}} =   \frac{\ket 0 + \ket 1}{\sqrt{2}}   
    \qquad \text{et} \qquad
    X \frac{\ket 0 - \ket 1}{\sqrt{2}} = - \frac{\ket 0 - \ket 1}{\sqrt{2}}
\end{equation*}

Probabilités:

\begin{equation*}
    p(+) = |\langle +|\psi\rangle|^2 = |\langle 0| H^\dagger |\psi\rangle|^2   \qquad \text{et} \qquad    p(-) = |\langle -|\psi\rangle|^2 = |\langle 1| H^\dagger |\psi\rangle|^2
\end{equation*}

Une mesure dans la base $X$ est équivalente à une mesure dans la base $Z$ après application de l'opérateur Hadamard sur l'état quantique.

En terme de projecteurs,

\begin{equation*}
    P_Z = |0\rangle \langle 0|  +  |1\rangle \langle 1|
\end{equation*}

\begin{equation*}
    P_X = |+\rangle \langle +|  +  |-\rangle \langle -| = H P_Z H^\dagger
\end{equation*}
-----------------------------

In [ ]:
from qiskit.quantum_info import Operator

H = Operator.from_label('H')

bob_base_bits = random.randint(0,2,N)
print(bob_base_bits)

bob_measurements = []

for i in range(N):
    q = bob_qubits[i]
    b = bob_base_bits[i]

    if b == 0:
        res = q.measure([0])
        measured_bit = int(res[0])
        bob_measurements.append(measured_bit)
    else:
        q.evolve(H)
        res = q.measure([0])
        measured_bit = int(res[0])
        bob_measurements.append(measured_bit)

for m in bob_measurements:
    print(m, end=' ')


**Alice:** Voilà les nombres aléatoires que j'ai utilisés

In [ ]:
print(alice_base_bits)

**Bob:** Bien reçu! Voici les miens:

In [ ]:
print(bob_base_bits)

**Alice et Bob:** Sélectionnons les bits représentant la même base!

(Et donc les qubits partageant la même base...)

In [ ]:
shared_base_bits_indices = []

for i in range(N):
    if alice_base_bits[i] == bob_base_bits[i]:
        shared_base_bits_indices.append(i)

print("Indices des bits partagés :", shared_base_bits_indices)

**Bob:** Je peux maintenant récupérer la clé générée par Alice parmi mes mesures.

In [ ]:
shared_data_bits = []

for idx in shared_base_bits_indices:
    data_bit = bob_measurements[idx]
    shared_data_bits.append(data_bit)

print(shared_data_bits)

Vérification: ce sont bien les données encodées par Alice: 

In [ ]:
shared_data_bits_original = []

for idx in shared_base_bits_indices:
    data_bit = alice_data_bits[idx]
    shared_data_bits_original.append(data_bit)

print(shared_data_bits_original)

----------------------------------------
### Attention!

**Ce n'est pas suffisant, discuter de l'interférence d'une tierce personne!**

La communication des qubits est certainement imparfaite, ce qui conduira naturellement à des clés certes légèrement différentes, mais quand même inutilisables! Pour remédier à ce problème, il existe des techniques de *information reconciliation* et *privacy amplification*.

Toutefois, il est d'abord nécessaire de vérifier la qualité du transfert d'information et la trace d'une **potentielle interférence par une personne tierce** en comparant une partie des bits partagés. Les bits restant constitueront la clé gardée secrète. 

**Eve:** 